# Run this notebook to use the model locally
You can pass inputs in the last cell

In [ ]:
import numpy as np
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
from sklearn.model_selection import train_test_split
from transformers import (
    DistilBertForSequenceClassification, 
    AutoTokenizer,
    Trainer, 
    TrainingArguments,
    EarlyStoppingCallback,
    AutoModelForSequenceClassification
)
import os
import json

# Set random seeds for reproducibility
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(RANDOM_SEED)

# Check for MPS (Apple Silicon) device
use_mps = torch.backends.mps.is_available()
try:
    if torch.backends.mps.is_available():
        device = torch.device("mps")
        print("Using MPS device")
    elif torch.cuda.is_available():
        device = torch.device("cuda")
        print("Using CUDA device")
    else:
        device = torch.device("cpu")
        print("Using CPU device")
except:
    device = torch.device("cpu")
    print("Fallback to CPU device due to error")

print(f"Using device: {device}")

# Model configuration with smaller memory footprint
MODEL_NAME = "distilbert-base-uncased"
NUM_LABELS = 2  # Binary classification
OUTPUT_DIR = "./results"
MODEL_DIR = "./saved_model"
MAX_LENGTH = 128  # Limit sequence length to save memory


MODEL_DIR = "./saved_model"
model = AutoModelForSequenceClassification.from_pretrained(MODEL_DIR)
tokenizer = AutoTokenizer.from_pretrained(MODEL_DIR)

Using MPS device
Using device: mps


In [ ]:
def predict_sentiment(text):
    # Process one example at a time
    inputs = tokenizer(
        text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=MAX_LENGTH,
        padding="max_length"
    )
    
    # Move inputs to the right device
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    # Use inference mode to save memory
    with torch.inference_mode():
        outputs = model(**inputs)
    
    logits = outputs.logits.cpu()  # Move back to CPU for post-processing
    prediction = torch.argmax(logits, dim=-1).item()
    
    return {
        "text": text,
        # "predicted_class": prediction,
        # "class_name": model.config.id2label[prediction],
        "offensive": bool(prediction),
        "confidence": torch.softmax(logits, dim=-1)[0][prediction].item()
    }

def print_prediction(pred): 
    print(json.dumps(pred, indent=4))
model = model.to(device)

In [15]:
test_input = """
I am not offensive!
""".strip()
print(json.dumps(predict_sentiment(test_input), indent=4))

{
    "text": "I am not offensive!",
    "offensive": false,
    "confidence": 0.9957155585289001
}
